In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import scipy
import scipy.special

In [10]:
# elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz", trainable=True)
# elmo = hub.Module("http://files.deeppavlov.ai/deeppavlov_data/elmo_ru-news_wmt11-16_1.5M_steps.tar.gz", trainable=True)
elmo = hub.Module("/home/alx/Cloud/spell_corr/py_spelling_corrector/bidirectional_lms/elmo_ru_news/tf_hub_model_epoch_n_3/", trainable=False)
base_path = "/home/alx/Cloud/spell_corr/py_spelling_corrector/bidirectional_lms/elmo_ru_news"
ckpt_prefixed_path = base_path + "/model.ckpt-0003"
metafile_path = base_path + "/model.ckpt-0003.meta"
# ckpt_prefixed_path = base_path + "/model.ckpt-1327437"
# metafile_path = base_path + "/model.ckpt-1327437.meta"

tf.train.list_variables(ckpt_prefixed_path)

softmax_w = tf.train.load_variable(ckpt_prefixed_path, 'lm/softmax/W')
# torch_w = torch.from_numpy(np.concatenate((softmax_w, softmax_w), axis=1))

softmax_bias = tf.train.load_variable(ckpt_prefixed_path, 'lm/softmax/b')

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
embeddings = elmo(["<S> <S> мера работы </S> </S>"], signature="default", as_dict=True)["lstm_outputs2"]
results = sess.run(embeddings)
results

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[[ 0.79889953,  0.5857542 ,  0.13159186, ..., -0.14127856,
          1.8105521 ,  0.48734403],
        [ 1.1598899 ,  0.23845488,  0.2947522 , ...,  0.12454441,
          1.2984084 ,  0.25909704],
        [ 1.7405832 , -0.9548652 , -0.7513319 , ...,  0.2896814 ,
         -0.03676649,  0.3693152 ],
        [ 1.6471536 ,  0.4259458 ,  0.07340121, ...,  0.8653351 ,
          0.09196165, -1.5240464 ],
        [ 1.3139178 ,  0.43473622, -0.7177054 , ..., -0.5992434 ,
          0.68112504,  0.16736832],
        [ 1.7894601 ,  0.35135365, -0.46799153, ..., -0.17237437,
          0.75451815,  0.37456733]]], dtype=float32)

In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
embeddings = elmo(["<S> мера работы </S>"], signature="default", as_dict=True)["lstm_outputs2"]
results = sess.run(embeddings)
results

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[[ 0.79889953,  0.5857542 ,  0.13159186, ...,  0.06740814,
          1.1231669 ,  0.18051529],
        [ 1.4343534 , -0.5682142 , -0.74894536, ...,  0.02373633,
         -0.34858432,  0.00790232],
        [ 1.4725636 ,  0.8675049 ,  0.2891655 , ...,  0.56168586,
          0.23248845, -1.624943  ],
        [ 1.2200443 ,  0.5059935 , -0.51519847, ..., -0.17237437,
          0.75451815,  0.37456733]]], dtype=float32)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
embeddings = elmo(["мера работы"], signature="default", as_dict=True)["lstm_outputs2"]
results = sess.run(embeddings)
results

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[[ 1.2291199 , -0.8567653 , -1.1947494 , ..., -0.11537272,
         -0.22501028, -0.32351553],
        [ 1.1092613 ,  0.3798915 , -0.11941656, ..., -0.04146627,
          0.07232222, -0.7936702 ]]], dtype=float32)

In [25]:
elmo.__dict__

{'_graph': <tensorflow.python.framework.ops.Graph at 0x7f430d785c88>,
 '_spec': <tensorflow_hub.native_module._ModuleSpec at 0x7f41b0375c18>,
 '_trainable': False,
 '_tags': set(),
 '_name': 'module_1',
 '_impl': <tensorflow_hub.native_module._ModuleImpl at 0x7f41b0375d30>}

In [6]:
results[0].shape

(4, 1024)

In [ ]:
######################################################

In [44]:
# new model from n3 tf hub checkpont:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
embeddings = elmo(["<S> мама мыла раму </S>"], signature="default", as_dict=True)["lstm_outputs2"]
results = sess.run(embeddings)

results

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[[ 0.79889953,  0.5857542 ,  0.13159186, ...,  0.4088966 ,
          1.5040114 , -0.448631  ],
        [ 1.1420202 , -0.683823  ,  0.4307257 , ...,  0.50999224,
         -0.19480745,  2.126551  ],
        [ 0.70192194,  0.10042723,  1.4029036 , ..., -0.17755914,
          0.2672583 ,  2.2676558 ],
        [ 1.1150804 ,  0.79951286, -0.13341913, ...,  0.6540302 ,
          0.21329385,  1.6829026 ],
        [ 1.1998848 ,  1.2393286 ,  0.2677026 , ..., -0.17237437,
          0.75451815,  0.37456733]]], dtype=float32)

In [58]:
# new model from n3 tf hub checkpont:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
tokens_input = [["<S>", "мама", "мыла", "раму", "</S>"], ["<S>", "мама", "</S>", "", ""]]
tokens_length = [5, 3]
embeddings = elmo(inputs={
        "tokens": tokens_input,
        "sequence_len": tokens_length
    }, signature="tokens", as_dict=True)["lstm_outputs2"]
results = sess.run(embeddings)

results

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


array([[[ 0.7988994 ,  0.585754  ,  0.1315921 , ...,  0.40889722,
          1.5040106 , -0.44863087],
        [ 1.14202   , -0.6838228 ,  0.43072546, ...,  0.509992  ,
         -0.19480795,  2.1265514 ],
        [ 0.70192134,  0.10042708,  1.4029037 , ..., -0.17755967,
          0.26725802,  2.2676554 ],
        [ 1.1150799 ,  0.79951274, -0.13341969, ...,  0.6540301 ,
          0.21329416,  1.6829026 ],
        [ 1.1998851 ,  1.2393286 ,  0.2677025 , ..., -0.17237389,
          0.75451815,  0.3745678 ]],

       [[ 0.7988994 ,  0.585754  ,  0.1315921 , ..., -0.13156727,
          1.6061943 , -0.6747255 ],
        [ 1.14202   , -0.6838228 ,  0.43072546, ...,  0.34244913,
         -0.48817977,  0.51071817],
        [ 0.6085679 ,  0.16516256, -0.12187518, ..., -0.17237389,
          0.75451815,  0.3745678 ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.  

In [59]:
results.shape

(2, 5, 1024)

In [42]:
left_results = np.dot(results[0][:,:512], softmax_w.transpose()) + softmax_bias
right_results = np.dot(results[0][:,512:], softmax_w.transpose()) + softmax_bias


In [33]:
softmax_bias

array([ 5.08596516e+00,  3.11770749e+00,  1.06373644e+01, ...,
       -1.04186125e-02, -1.05455350e-02, -9.74634849e-03], dtype=float32)

In [40]:
softmax_bias

array([ 5.10069561e+00,  3.15454316e+00,  1.06414719e+01, ...,
       -1.04186125e-02, -1.05455350e-02, -9.74634849e-03], dtype=float32)

In [38]:
softmax_bias.shape

(1000000,)

In [29]:
softmax_w.shape

(1000000, 512)

In [30]:
left_results = np.dot(results[0][:,:512], softmax_w.transpose()) + softmax_bias
right_results = np.dot(results[0][:,512:], softmax_w.transpose()) + softmax_bias

right_results

array([[ 5.3856764 ,  2.597848  , 11.721089  , ...,  0.1906159 ,
        -1.9993918 , -1.0631936 ],
       [ 4.8413115 ,  1.9764745 , 11.738583  , ..., -0.02939369,
        -1.0792421 , -1.8582284 ],
       [ 5.2286363 ,  2.9957576 , 11.781627  , ..., -0.4307802 ,
        -1.2651947 , -1.4347929 ],
       [ 4.8418155 ,  4.24338   , 11.911145  , ...,  1.7320998 ,
        -2.556269  , -1.1907287 ],
       [ 5.752591  ,  2.1912894 , 10.8296795 , ..., -0.02898552,
        -0.69286543, -0.20646814]], dtype=float32)

In [31]:
right_probas = scipy.special.softmax(right_results, axis=1)
left_probas = scipy.special.softmax(left_results, axis=1)
print(left_probas)
print(right_probas)

[[4.1389885e-05 1.2339567e-06 1.4784723e-02 ... 2.9702841e-07
  3.6191526e-07 3.1415297e-07]
 [1.0177101e-04 2.8236516e-05 4.0044580e-02 ... 9.1196756e-07
  1.8523893e-07 5.9393534e-07]
 [1.1864878e-04 1.2445122e-05 7.3311411e-02 ... 2.1117930e-06
  5.8212385e-07 2.7331825e-07]
 [3.9510931e-05 2.7510239e-06 5.9731834e-02 ... 8.7714938e-07
  1.3708799e-07 1.1330882e-07]
 [3.8755370e-05 2.9163414e-07 1.3641552e-02 ... 1.1782676e-06
  2.7666403e-07 4.3078214e-07]]
[[1.00687364e-04 6.19778439e-06 5.68080470e-02 ... 5.58198622e-07
  6.24713223e-08 1.59318589e-07]
 [4.89340819e-05 2.78887251e-06 4.84237298e-02 ... 3.75224232e-07
  1.31325137e-07 6.02612502e-08]
 [9.02637476e-05 9.67802771e-06 6.33054599e-02 ... 3.14528592e-07
  1.36545850e-07 1.15245300e-07]
 [4.52710883e-05 2.48842280e-05 5.32098524e-02 ... 2.01970852e-06
  2.77251306e-08 1.08622729e-07]
 [1.86557867e-04 5.29858198e-06 2.99064610e-02 ... 5.75316108e-07
  2.96201591e-07 4.81755649e-07]]


In [ ]:
##############

In [35]:
right_results = np.dot(results[0][:,512:], softmax_w.transpose()) + softmax_bias
left_results = np.dot(results[0][:,:512], softmax_w.transpose()) + softmax_bias
right_results

array([[ 5.467596  ,  2.696701  , 11.68482   , ...,  0.1906159 ,
        -1.9993918 , -1.0631936 ],
       [ 4.7651844 ,  2.0939171 , 11.687746  , ..., -0.02939369,
        -1.0792421 , -1.8582284 ],
       [ 5.226398  ,  3.041084  , 11.739252  , ..., -0.4307802 ,
        -1.2651947 , -1.4347929 ],
       [ 5.106469  ,  4.49094   , 11.897094  , ...,  1.7320998 ,
        -2.556269  , -1.1907287 ],
       [ 5.820825  ,  2.0943055 , 10.80845   , ..., -0.02898552,
        -0.69286543, -0.20646814]], dtype=float32)

In [36]:
right_probas = scipy.special.softmax(right_results, axis=1)
left_probas = scipy.special.softmax(left_results, axis=1)
print(left_probas)
print(right_probas)

[[4.6148794e-05 2.7521944e-06 1.4753032e-02 ... 2.9745334e-07
  3.6243298e-07 3.1460240e-07]
 [6.3400032e-05 1.5464353e-05 3.9381847e-02 ... 9.1149451e-07
  1.8514285e-07 5.9362725e-07]
 [9.9187535e-05 1.2033122e-05 7.0392981e-02 ... 2.1155606e-06
  5.8316232e-07 2.7380585e-07]
 [3.8434708e-05 3.9461506e-06 5.7148423e-02 ... 8.7893295e-07
  1.3736675e-07 1.1353922e-07]
 [4.3166594e-05 6.6936070e-07 1.3724885e-02 ... 1.1788003e-06
  2.7678911e-07 4.3097691e-07]]
[[1.09504705e-04 6.85564555e-06 5.48957698e-02 ... 5.59331511e-07
  6.25981116e-08 1.59641928e-07]
 [4.54085020e-05 3.14065664e-06 4.60858420e-02 ... 3.75731986e-07
  1.31502844e-07 6.03427992e-08]
 [9.00764571e-05 1.01284131e-05 6.06886782e-02 ... 3.14578983e-07
  1.36567735e-07 1.15263759e-07]
 [5.90538657e-05 3.19100473e-05 5.25265820e-02 ... 2.02198635e-06
  2.77564016e-08 1.08745247e-07]
 [1.99889095e-04 4.81263169e-06 2.93013193e-02 ... 5.75769491e-07
  2.96434990e-07 4.82135306e-07]]


In [ ]:
######################

In [47]:
right_results = np.dot(results[0][:,512:], softmax_w.transpose()) + softmax_bias
left_results = np.dot(results[0][:,:512], softmax_w.transpose()) + softmax_bias
right_results

array([[17.464626  ,  9.964719  , 15.220111  , ...,  1.4305799 ,
        -0.7833629 ,  0.03000938],
       [17.373268  , 10.2835655 , 14.385449  , ..., -2.2208967 ,
         0.4043692 , -0.725305  ],
       [14.959097  ,  9.207074  , 14.698032  , ..., -1.966989  ,
        -1.0873272 , -2.4141622 ],
       [10.921311  ,  9.388043  , 15.45725   , ...,  0.03011085,
        -0.38393864, -0.27609923],
       [12.527948  ,  7.7219534 , 13.72964   , ..., -0.36991668,
        -1.2488997 , -0.38569853]], dtype=float32)

In [48]:
right_probas = scipy.special.softmax(right_results, axis=1)
left_probas = scipy.special.softmax(left_results, axis=1)
print(left_probas)
print(right_probas)

[[3.13395489e-04 8.63772154e-01 7.54667539e-03 ... 3.72587294e-09
  1.03295017e-08 9.34191746e-09]
 [6.21227664e-04 4.59137000e-02 9.44168773e-03 ... 1.60676583e-09
  1.06974545e-08 2.18480745e-09]
 [4.46387538e-04 4.93659005e-02 1.32690258e-02 ... 6.77843670e-10
  2.09768221e-08 2.24511565e-09]
 [1.73898021e-04 5.12293518e-01 9.95577034e-03 ... 7.54184215e-10
  6.54506227e-09 1.20851840e-09]
 [5.65318478e-05 9.29430664e-01 4.79735201e-03 ... 6.69970857e-10
  4.41607639e-09 1.80505433e-09]]
[[6.6016716e-01 3.6516192e-04 6.9963776e-02 ... 7.1805239e-08
  7.8460793e-09 1.7696866e-08]
 [2.3888291e-01 1.9914360e-04 1.2039041e-02 ... 7.3883422e-10
  1.0202015e-08 3.2966618e-09]
 [4.2894501e-02 1.3624792e-04 3.3038683e-02 ... 1.9120343e-09
  4.6081632e-09 1.2226195e-09]
 [6.2493677e-04 1.3487959e-04 5.8313493e-02 ... 1.1637186e-08
  7.6918134e-09 8.5676568e-09]
 [3.2812104e-02 2.6842128e-04 1.0912446e-01 ... 8.2141440e-08
  3.4105550e-08 8.0855315e-08]]


In [60]:
right_probas.shape

(5, 1000000)

In [62]:
np.array([left_probas, right_probas])

(2, 5, 1000000)

In [64]:
np.swapaxes(np.array([left_probas, right_probas]), 0,1).shape

(5, 2, 1000000)

In [65]:
from language_models.elmolm_tfhub import ELMOLMTFHub
elmolm_tfhub = ELMOLMTFHub()

In [66]:
import pprint
import numpy as np
from spelling_correction_models.elmo_40in_spelling_corrector.elmo_40in2_spelling_corrector import ELMO40in2SpellingCorrector
from copy import deepcopy
from spelling_correction_models.elmo_40in_spelling_corrector.helper_fns import *


/home/alx/Cloud/spell_corr/py_spelling_corrector


In [67]:
elmo40in = ELMO40in2SpellingCorrector(mini_batch_size=10, language_model=elmolm_tfhub)
# sccg = elmo40in.sccg

Init LetterCaser.
Init language_model.
Init spelling_correction_candidates_generator.
Initialization Completed.


In [68]:
elmo40in.prepare_analysis_dict_for_sentence("что-нить наидем")

2019-10-30 16:03:44.607 WARNING in 'tensorflow'['deprecation_wrapper'] at line 119: From /home/alx/Cloud/spell_corr/py_spelling_corrector/language_models/elmolm_tfhub.py:71: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



2019-10-30 16:03:44.629 WARNING in 'tensorflow'['deprecation_wrapper'] at line 119: From /home/alx/Cloud/spell_corr/py_spelling_corrector/language_models/elmolm_tfhub.py:72: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2019-10-30 16:03:52.138 INFO in 'tensorflow'['saver'] at line 1499: Saver not created because there are no variables in the graph to restore


{'input_sentence': 'что-нить наидем',
 'tokenized_input_sentence': ['<S>', 'что-нить', 'наидем', '</S>'],
 'word_substitutions_candidates': [{'tok_idx': 0, 'top_k_candidates': []},
  {'tok_idx': 1,
   'top_k_candidates': [{'lm_advantage': 9.310109086646285,
     'advantage': 5.710109086646286,
     'token_str': 'что-нибудь',
     'zero_hypothesis': False,
     'error_score': -3.6,
     'token_merges': 0,
     'token_splits': None},
    {'lm_advantage': 3.7849889282795726,
     'advantage': -0.21501107172042744,
     'token_str': 'что нить',
     'zero_hypothesis': False,
     'error_score': -4.0,
     'token_merges': 0,
     'token_splits': 1},
    {'lm_advantage': 0.0,
     'advantage': -1.0,
     'token_str': 'что-нить',
     'zero_hypothesis': True,
     'error_score': 0.0,
     'token_merges': 0,
     'token_splits': None}]},
  {'tok_idx': 2,
   'top_k_candidates': [{'lm_advantage': 6.087318654180866,
     'advantage': 3.327318654180866,
     'token_str': 'на идем',
     'zero_hypo